In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sqlite3
import pandas as pd
import yaml
import json
import edurel.utils.sqlite as slu


DB_DIR = "/home/iclassen/1/Spider2/spider2-lite/resource/databases/spider2-localdb"

In [12]:
con = slu.file_con(f"{DB_DIR}/Db-IMDB.sqlite")

In [14]:
sql = "select * from movie limit 2"
slu.sql_df(con, sql)

,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861


In [15]:
slu.schema_print(con)

Table: Movie (index INTEGER NULL, MID TEXT NULL, title TEXT NULL, year TEXT NULL, rating REAL NULL, num_votes INTEGER NULL)
Table: Genre (index INTEGER NULL, Name TEXT NULL, GID INTEGER NULL)
Table: Language (index INTEGER NULL, Name TEXT NULL, LAID INTEGER NULL)
Table: Country (index INTEGER NULL, Name TEXT NULL, CID INTEGER NULL)
Table: Location (index INTEGER NULL, Name TEXT NULL, LID INTEGER NULL)
Table: M_Location (index INTEGER NULL, MID TEXT NULL, LID REAL NULL, ID INTEGER NULL)
Table: M_Country (index INTEGER NULL, MID TEXT NULL, CID REAL NULL, ID INTEGER NULL)
Table: M_Language (index INTEGER NULL, MID TEXT NULL, LAID INTEGER NULL, ID INTEGER NULL)
Table: M_Genre (index INTEGER NULL, MID TEXT NULL, GID INTEGER NULL, ID INTEGER NULL)
Table: Person (index INTEGER NULL, PID TEXT NULL, Name TEXT NULL, Gender TEXT NULL)
Table: M_Producer (index INTEGER NULL, MID TEXT NULL, PID TEXT NULL, ID INTEGER NULL)
Table: M_Director (index INTEGER NULL, MID TEXT NULL, PID TEXT NULL, ID INTEGE

In [ ]:
local099_q = """
I need you to look into the actor collaborations and tell me 
how many actors have made more films with Yash Chopra 
than with any other director. 
This will help us understand his influence on the industry better.
"""

local099_sql = """
WITH YASH_CHOPRAS_PID AS (
    SELECT
        TRIM(P.PID) AS PID
    FROM
        Person P
    WHERE
        TRIM(P.Name) = 'Yash Chopra'
),
NUM_OF_MOV_BY_ACTOR_DIRECTOR AS (
    SELECT
        TRIM(MC.PID) AS ACTOR_PID,
        TRIM(MD.PID) AS DIRECTOR_PID,
        COUNT(DISTINCT TRIM(MD.MID)) AS NUM_OF_MOV
    FROM
        M_Cast MC
    JOIN
        M_Director MD ON TRIM(MC.MID) = TRIM(MD.MID)
    GROUP BY
        ACTOR_PID,
        DIRECTOR_PID
),
NUM_OF_MOVIES_BY_YC AS (
    SELECT
        NM.ACTOR_PID,
        NM.DIRECTOR_PID,
        NM.NUM_OF_MOV AS NUM_OF_MOV_BY_YC
    FROM
        NUM_OF_MOV_BY_ACTOR_DIRECTOR NM
    JOIN
        YASH_CHOPRAS_PID YCP ON NM.DIRECTOR_PID = YCP.PID
),
MAX_MOV_BY_OTHER_DIRECTORS AS (
    SELECT
        ACTOR_PID,
        MAX(NUM_OF_MOV) AS MAX_NUM_OF_MOV
    FROM
        NUM_OF_MOV_BY_ACTOR_DIRECTOR NM
    JOIN
        YASH_CHOPRAS_PID YCP ON NM.DIRECTOR_PID <> YCP.PID
    GROUP BY
        ACTOR_PID
),
ACTORS_MOV_COMPARISION AS (
    SELECT
        NMY.ACTOR_PID,
        CASE WHEN NMY.NUM_OF_MOV_BY_YC > IFNULL(NMO.MAX_NUM_OF_MOV, 0) THEN 'Y' ELSE 'N' END AS MORE_MOV_BY_YC
    FROM
        NUM_OF_MOVIES_BY_YC NMY
    LEFT OUTER JOIN
        MAX_MOV_BY_OTHER_DIRECTORS NMO ON NMY.ACTOR_PID = NMO.ACTOR_PID
)
SELECT
    COUNT(DISTINCT TRIM(P.PID)) AS "Number of actor"
FROM
    Person P
WHERE
    TRIM(P.PID) IN (
        SELECT
            DISTINCT ACTOR_PID
        FROM
            ACTORS_MOV_COMPARISION
        WHERE
            MORE_MOV_BY_YC = 'Y'
    );

"""

In [ ]:
slu.sql_df(con, local099_sql)


,Number of actor
0,107


# log

In [18]:
con_log = slu.file_con(f"{DB_DIR}/log.sqlite")


In [19]:
slu.schema_print(con_log)

Table: mst_users (user_id varchar(255) NULL, sex varchar(255) NULL, birth_date varchar(255) NULL, register_date varchar(255) NULL, register_device varchar(255) NULL, withdraw_date varchar(255) NULL)
Table: action_log (session varchar(255) NULL, user_id varchar(255) NULL, action varchar(255) NULL, category varchar(255) NULL, products varchar(255) NULL, amount INTEGER NULL, stamp varchar(255) NULL)
Table: activity_log (stamp varchar(255) NULL, session varchar(255) NULL, action varchar(255) NULL, option varchar(255) NULL, path varchar(255) NULL, search_type varchar(255) NULL)
Table: read_log (stamp varchar(255) NULL, session varchar(255) NULL, action varchar(255) NULL, url varchar(255) NULL)
Table: form_log (stamp varchar(255) NULL, session varchar(255) NULL, action varchar(255) NULL, path varchar(255) NULL, status varchar(255) NULL)
Table: form_error_log (stamp varchar(255) NULL, session varchar(255) NULL, form varchar(255) NULL, field varchar(255) NULL, error_type varchar(255) NULL, val